In [ ]:
#import libraries
import pandas as pd
import numpy as np
import time
import collections
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import nltk
import ssl
import re
import gc
import sklearn
from sklearn import feature_selection, feature_extraction, naive_bayes, pipeline, metrics

#SSL error fix
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

#download corpuses
nltk.download('stopwords')
nltk.download('omw-1.4')

In [ ]:
#load data using only 'subreddit' and 'body' features
start = time.process_time()
cols = ['subreddit', 'body ']
df = pd.read_csv('politics25.tsv', nrows=20000000, sep='\t', usecols=cols)
df = df.dropna()
end = time.process_time()
print(end-start)

In [ ]:
#preprocessing
df = df.rename(columns={'body ':'body'})

#drop nontrivial subreddits
far_left = ['chapotraphouse', 'anarchism', 'completeanarchy']
drops = ['antiwork', 'keep_track', 'jordanpeterson','political_revolution',
         'esist', 'politics', 'politicalhumor', 'anarcho_capitalism', 'cringeanarchy',
         'selfawarewolves', 'the_mueller' , 'onguardforthee']
drops = far_left + drops
df = df[~df.subreddit.isin(drops)]

#find large (>100 0000 posts) subreddits 
large_subreddits = df['subreddit'].value_counts()[df['subreddit']
                    .value_counts()>100000].index.tolist()

#drop large subreddits
df_small = df[~df.subreddit.isin(large_subreddits)]

#sample large subreddits to 100 000 posts
for sub in large_subreddits:
    sample = df[df.subreddit == sub].sample(100000)
    df_small = df_small.append(sample)
    
#rename column
df_small = df.rename(columns={'subreddit': 'spectrum'})
    
#rename subreddits based on political spectrum
left = ['liberal', 'wayofthebern', 'ourpresident', 'libertarian',
        'sandersforpresident', 'socialism', 'latestagecapitalism',
        'fullcommunism', 'communism', 'neoliberal', 'goldandblack',
        'progressive', 'latestagesocialism']
right = ['conservative', 'republican', 'the_donald', 'thenewright']

df_small['spectrum'] = df_small['spectrum'].replace(left, 'left')
df_small['spectrum'] = df_small['spectrum'].replace(right, 'right')

#sample spectrums
df_spectrum = pd.DataFrame()
for spec in ['left', 'right']:
    sample = df_small[df_small.spectrum == spec].sample(100000)
    df_spectrum = df_spectrum.append(sample)


In [ ]:
#show distribution of data
fig, ax = plt.subplots()
fig.suptitle("y", fontsize=12)
df_spectrum["spectrum"].reset_index().groupby("spectrum").count().sort_values(by= 
       "index").plot(kind="barh", legend=False, 
        ax=ax).grid(axis='x')
plt.show()

<font size="7">TF-IDF</font>

In [ ]:
#preprocessing function
def preprocess(text, lemm=True, stopwords=None):
    
    #clean
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    #tokenize and remove stopwords
    lst_text = text.split()
    if stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    stopwords]
                
    # if lemmatise
    if lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    # return string
    text = " ".join(lst_text)
    return text

In [ ]:
#define stopwords
stopwords = nltk.corpus.stopwords.words("english")

#process text
start = time.process_time()
df_spectrum['processed'] = df_spectrum['body'].apply(lambda x: 
                preprocess(x, lemm=True,
                stopwords=stopwords))
end = time.process_time()
print(end-start)

In [ ]:
#save processed data to speed up future computations
df_spectrum.to_csv('spectrum.csv')

#load preprocessed data
#df_spectrum = pd.read_csv('spectrum.csv')
#df_spectrum = df_spectrum.dropna()

In [ ]:
#create TF-IDF
start = time.process_time()

#split data 
train, test = sklearn.model_selection.train_test_split(df_spectrum, test_size=0.3)

#target
y_train = train['spectrum'].values
y_test = test['spectrum'].values

#Tf-idf
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_features=10000, ngram_range=(1,2))

#feature matrix
vectorizer.fit(train['processed'])
X_train = vectorizer.transform(train['processed'])
vocab = vectorizer.vocabulary_

end = time.process_time()
print(end-start)

In [ ]:
#classifier
start = time.process_time()
classifier = naive_bayes.MultinomialNB()
#classifier = KMeans(n_clusters=2).fit(X_train)

#pipeline
model = pipeline.Pipeline([("vectorizer", vectorizer),  
                           ("classifier", classifier)])
#train classifier
model["classifier"].fit(X_train, y_train)

#test
X_test = test["processed"].values
predicted = model.predict(X_test)
predicted_prob = model.predict_proba(X_test)

end = time.process_time()
print(end-start)

In [ ]:
#print results

classes = np.unique(y_test)
y_test_array = pd.get_dummies(y_test, drop_first=False).values
    
## Accuracy, Precision, Recall
accuracy = metrics.accuracy_score(y_test, predicted)
auc = metrics.roc_auc_score(y_test, predicted_prob[:,1])
                            #multi_class="ovr")
print("Accuracy:",  round(accuracy,2))
print("Auc:", round(auc,2))
print("Detail:")
print(metrics.classification_report(y_test, predicted))

fig, ax = plt.subplots(nrows=1, ncols=2)
## Plot roc
for i in range(len(classes)):
    fpr, tpr, thresholds = metrics.roc_curve(y_test_array[:,i],  
                           predicted_prob[:,i])
    ax[0].plot(fpr, tpr, lw=3, 
              label='{0} (area={1:0.2f})'.format(classes[i], 
                              metrics.auc(fpr, tpr))
               )
ax[0].plot([0,1], [0,1], color='navy', lw=3, linestyle='--')
ax[0].set(xlim=[-0.05,1.0], ylim=[0.0,1.05], 
          xlabel='False Positive Rate', 
          ylabel="True Positive Rate (Recall)", 
          title="Receiver operating characteristic")
ax[0].legend(loc="lower right")
ax[0].grid(True)
    
## Plot precision-recall curve
for i in range(len(classes)):
    precision, recall, thresholds = metrics.precision_recall_curve(
                 y_test_array[:,i], predicted_prob[:,i])
    ax[1].plot(recall, precision, lw=3, 
               label='{0} (area={1:0.2f})'.format(classes[i], 
                                  metrics.auc(recall, precision))
              )
ax[1].set(xlim=[0.0,1.05], ylim=[0.0,1.05], xlabel='Recall', 
          ylabel="Precision", title="Precision-Recall curve")
ax[1].legend(loc="best")
ax[1].grid(True)
plt.show()